In [ ]:
# default_exp transform

In [ ]:
# export
import torch

In [ ]:
# export
def deg2rad(angles):
    if torch.__version__ >= '1.6':
        return torch.deg2rad(angles)

    M_PI_180 = torch.tensor(0.017453292519943295769236907684886127134428718885417, dtype=torch.float64)
    return torch.mul(angles, M_PI_180.to(angles.device))

In [ ]:
# export
def axis_angle_rotation(axis: str, angle, mode='normal'):
    """
    Return the rotation matrices for one of the rotations about an axis
    of which Euler angles describe, for each value of the angle given.

    Args:
        axis: Axis label "X" or "Y or "Z".
        angle: any shape tensor of Euler angles in radians

    Returns:
        Rotation matrices as tensor of shape (..., 3, 3).
    """

    cos = torch.cos(angle).to(angle.device)
    sin = torch.sin(angle).to(angle.device)
    one = torch.ones_like(angle).to(angle.device)
    zero = torch.zeros_like(angle).to(angle.device)
    
    if mode == 'normal':
        ff = 1
    elif mode == '3ddfa':
        ff = -1

    if axis == "X":
        R_flat = (one, zero, zero, 
                  zero, cos, -sin*ff,
                  zero, sin*ff, cos)
    if axis == "Y":
        R_flat = (cos, zero, sin*ff,
                  zero, one, zero,
                  -sin*ff, zero, cos)
    if axis == "Z":
        R_flat = (cos, -sin*ff, zero,
                  sin*ff, cos, zero,
                  zero, zero, one)

    return torch.stack(R_flat, -1).reshape(angle.shape + (3, 3))

def angle2matrix(angles, mode='normal'):
    angles = angles.double()
    if mode == 'normal':
        matrices = map(axis_angle_rotation, "ZYX", torch.unbind(deg2rad(angles), -1)[::-1], [mode]*3)
    elif mode =='3ddfa':
        matrices = map(axis_angle_rotation, "XYZ", torch.unbind(angles, -1), [mode]*3)
    return functools.reduce(torch.matmul, matrices).float()

In [ ]:
# export
def similarity_transform(verts, s, R, t3d):
    tfm_verts = s[..., None] * verts.matmul(R.transpose(-1, -2)) + t3d.unsqueeze(-2)
    return tfm_verts

def transform(vertices, s, angles, t3d, mode='normal'):
    R = angle2matrix(angles, mode)
    return similarity_transform(vertices.float(), s, R, t3d)

In [ ]:
# export
def denormal_mesh(verts, mean=None, std=None, pose=None):
    if std is not None:
        verts *= std
    if mean is not None:
        verts += mean
    if pose is not None:
        s = pose[6]/1000/0.0012
        angles = pose[:3]
        t = pose[3:6]*225
        verts = transform(verts, s, angles, t, '3ddfa')
    return verts

In [ ]:
from nbdev.export import *

notebook2script()

Converted index.ipynb.
Converted io.ipynb.
Converted transform.ipynb.
Converted utils.ipynb.
